In [59]:
# Correcting the initial generated file


def process_file(input_filename, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(output_filename, 'w', encoding='utf-8') as file:
        for line in lines:
            if line.strip() == '"':
                file.write("\n")  # write a newline for lines that are just a quote
            elif line.strip() == '",O':
                continue
            else:
                corrected_line = line.replace(',', '\t')
                file.write(corrected_line)

process_file('data/ner/tokens_IOB.csv', 'data/ner/tokens_IOB_correct.csv')

In [60]:
import random

def read_and_split_data(input_filename, train_ratio=0.8, dev_ratio=0.1, test_ratio=0.1):
    # Ensure the ratios sum to 1
    assert train_ratio + dev_ratio + test_ratio == 1
    
    # Read and process the file
    with open(input_filename, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Split content into sentences based on newlines
    sentences = content.strip().split('\n\n')

    # Shuffle the sentences randomly for a fair split
    random.shuffle(sentences)

    # Calculate split indices
    total = len(sentences)
    train_end = int(total * train_ratio)
    dev_end = train_end + int(total * dev_ratio)
    
    # Create splits
    train_sentences = sentences[:train_end]
    dev_sentences = sentences[train_end:dev_end]
    test_sentences = sentences[dev_end:]

    # Save each split to a different file
    with open('data/ner/splits/train.csv', 'w', encoding='utf-8') as file:
        file.write('\n\n'.join(train_sentences))
    with open('data/ner/splits/dev.csv', 'w', encoding='utf-8') as file:
        file.write('\n\n'.join(dev_sentences))
    with open('data/ner/splits/test.csv', 'w', encoding='utf-8') as file:
        file.write('\n\n'.join(test_sentences))

read_and_split_data('data/ner/tokens_IOB_correct.csv')

In [61]:
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n\n')  # Splitting data by double newlines

    formatted_data = []
    for record in data:
        text, entities = '', []
        lines = record.split('\n')
        start = 0  # Initialize start of the first token
        entity_start = None  # Initialize the start of an entity
        entity_type = None  # Initialize the entity type
        for line in lines:
            parts = line.split('\t')
            if len(parts) >= 2:  # Ensure there's at least a token and a tag
                token, tag = parts[0], parts[1]
                # Update text with token followed by space
                text += token + ' '
                end = start + len(token)  # End index of the current token
                if tag.startswith('B-') or tag.startswith('I-'):  # Check if the tag indicates an entity
                    if tag.startswith('B-'):
                        # If there was a previous entity, append it
                        if entity_start is not None:
                            entities.append((entity_start, prev_end, entity_type))
                        # Start new entity
                        entity_start = start
                        entity_type = tag.split('-')[1]
                    if tag.startswith('I-') and entity_start is not None:
                        # Continue the entity
                        entity_type = tag.split('-')[1]  # Update the type if needed (typically not necessary)
                else:
                    # If ending part of entity, append it
                    if entity_start is not None:
                        entities.append((entity_start, end, entity_type))
                        entity_start = None  # Reset for the next entity

                # Prepare for next token
                start = end + 1
                prev_end = end  # Remember the end of the last token

        # Check for any trailing entity at the end
        if entity_start is not None:
            entities.append((entity_start, prev_end, entity_type))

        # Append the parsed data if there are entities
        if entities:
            formatted_data.append((text.strip(), {'entities': entities}))

    return formatted_data

# Load the data
train_data = load_data('data/ner/splits/train.csv')
dev_data = load_data('data/ner/splits/dev.csv')
test_data = load_data('data/ner/splits/test.csv')


In [62]:
train_data

[('Графіті № 4279', {'entities': [(0, 9, 'inscription')]}),
 ('Графіті № 569 табл LІІІ 3', {'entities': [(0, 9, 'inscription')]}),
 ('Опис Поверхня стіни в місці розташування напису сильно пошкоджена внас-',
  {'entities': [(0, 13, 'inscription'), (41, 54, 'inscription')]}),
 ('Опис На фресці прокреслені літери Г\ue01e що є традиційним скороченням слова-',
  {'entities': [(0, 7, 'inscription')]}),
 ('Розташування Висота від основи 172 см',
  {'entities': [(0, 19, 'inscription')]}),
 ('Категорія Молитовний напис', {'entities': [(21, 26, 'inscription')]}),
 ('З розбивкою на слова та відновленням втрачених фрагментів текст виглядає',
  {'entities': [(15, 23, 'inscription')]}),
 ('Датування Друга чверть кінець XI ст. за особливостями розташування',
  {'entities': [(0, 15, 'inscription')]}),
 ('Опис Рахункова частина однорядкового запису обчислення кількості проведе-',
  {'entities': [(0, 14, 'inscription')]}),
 ('Опис На фресці подвійними рисками прокреслений малюнок чотириконечного',
  {'

# Prepare SpaCy NER model

In [63]:
import spacy
from spacy.training import Example

# https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

# Load a blank model or a pre-existing model for your language
nlp = spacy.blank('uk')  # Replace 'en' with your language code if different

# Create the NER component if it doesn't exist
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

# Add labels
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# Train a SpaCy NER model

In [64]:
from spacy.util import minibatch, compounding

optimizer = nlp.initialize()

# Use the training data
for epoch in range(25):
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data, size=compounding(4., 32., 1.001))
    for batch in batches:
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)
    print(f"Losses at epoch {epoch}: {losses}")


Losses at epoch 0: {'ner': 847.1325059377859}
Losses at epoch 1: {'ner': 197.77414959095066}
Losses at epoch 2: {'ner': 167.21364815666143}
Losses at epoch 3: {'ner': 129.48124263491837}
Losses at epoch 4: {'ner': 86.00351586744404}
Losses at epoch 5: {'ner': 102.19373285152719}
Losses at epoch 6: {'ner': 73.09502306728628}
Losses at epoch 7: {'ner': 75.96652372289063}
Losses at epoch 8: {'ner': 40.98569605396566}
Losses at epoch 9: {'ner': 50.722432533328536}
Losses at epoch 10: {'ner': 43.74170506549736}
Losses at epoch 11: {'ner': 44.778562472894265}
Losses at epoch 12: {'ner': 45.84478987730159}
Losses at epoch 13: {'ner': 38.66052650295007}
Losses at epoch 14: {'ner': 45.193147625705976}
Losses at epoch 15: {'ner': 46.36258911422618}
Losses at epoch 16: {'ner': 23.040358080078025}
Losses at epoch 17: {'ner': 28.26308032807755}
Losses at epoch 18: {'ner': 15.244561014593867}
Losses at epoch 19: {'ner': 18.67243469446892}
Losses at epoch 20: {'ner': 24.27045718831691}
Losses at epoc

# Evaluation on the test data

## Prepare test data predictions

In [65]:
def transform_to_iob(nlp_model, data):
    transformed_data = []
    for text, annot in data:
        doc = nlp_model(text)
        # Extract IOB tokens from the SpaCy doc
        iob_tokens = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_type_ else t.ent_iob_ for t in doc]
        transformed_data.append(iob_tokens)
    return transformed_data

predictions_iob = transform_to_iob(nlp, test_data)


In [66]:
predictions_iob[:3]

[['B-inscription',
  'I-inscription',
  'O',
  'B-inscription',
  'I-inscription',
  'O',
  'B-inscription'],
 ['O', 'O', 'B-inscription'],
 ['B-inscription', 'I-inscription', 'O', 'O', 'O', 'O']]

## Prepare the true test data (groundtruth)

In [67]:
def transform_annotations_to_iob(data, nlp):
    iob_annotations = []

    for text, annot in data:
        doc = nlp.make_doc(text)  # Create a doc from text to ensure correct offsets
        tags = ['O'] * len(doc)  # Initialize tags with 'O'

        entities = annot['entities']
        # Convert character offsets to token offsets
        for start_char, end_char, label in entities:
            start_token, end_token = None, None
            for token_index, token in enumerate(doc):
                # Check if we've found the start token
                if token.idx == start_char:
                    start_token = token_index
                # Check if the current token extends beyond the start of the end_char range
                if token.idx + len(token) == end_char:
                    end_token = token_index
                # If the token encompasses the end character
                if token.idx + len(token) > end_char and end_token is None:
                    end_token = token_index

            if start_token is not None and end_token is not None:
                tags[start_token] = f"B-{label}"
                for i in range(start_token + 1, end_token + 1):
                    tags[i] = f"I-{label}"

        iob_annotations.append(tags)
    return iob_annotations

references_iob = transform_annotations_to_iob(test_data, nlp)

## Results evaluation

In [71]:
from seqeval.metrics import classification_report, accuracy_score, f1_score

# Assuming predictions_iob and references_iob are ready
results = classification_report(references_iob, predictions_iob, digits=4)
accuracy = accuracy_score(references_iob, predictions_iob)
f1 = f1_score(references_iob, predictions_iob)

print("Classification Report:\n", results)
print("Accuracy Score:", accuracy)
print("F1 Score:", f1 * 100.0, "%")

Classification Report:
               precision    recall  f1-score   support

 inscription     0.9771    0.9953    0.9861       214

   micro avg     0.9771    0.9953    0.9861       214
   macro avg     0.9771    0.9953    0.9861       214
weighted avg     0.9771    0.9953    0.9861       214

Accuracy Score: 0.9964136282127914
F1 Score: 98.61111111111111 %


# Example - how to use it

In [73]:
text = "мурування апсиди ХІІ ст. г давні ХІ ст. пройми арок галереї закла-."
doc = nlp(text)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('мурування апсиди', 'inscription')]


In [75]:
for text, _ in test_data[:5]:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc if 'inscription' in t.ent_type_])

Entities [('Опис На', 'inscription'), ('прокреслений простий', 'inscription'), ('хрест', 'inscription')]
Tokens [('Опис', 'inscription', 3), ('На', 'inscription', 1), ('прокреслений', 'inscription', 3), ('простий', 'inscription', 1), ('хрест', 'inscription', 3)]
Entities [('напис', 'inscription')]
Tokens [('напис', 'inscription', 3)]
Entities [('Розташування Висота', 'inscription')]
Tokens [('Розташування', 'inscription', 3), ('Висота', 'inscription', 1)]
Entities [('Опис', 'inscription'), ('Текстова частина', 'inscription')]
Tokens [('Опис', 'inscription', 3), ('Текстова', 'inscription', 3), ('частина', 'inscription', 1)]
Entities [('слова та', 'inscription')]
Tokens [('слова', 'inscription', 3), ('та', 'inscription', 1)]
